In [ ]:
import os
import math
import random
import shutil
import scipy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

import tensorflow as tf
from keras import Model
from keras.layers import GlobalAveragePooling2D
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

In [ ]:
BASE_DIR = 'data/'
OG_DATA_DIR = 'data/64/'
subfolders = next(os.walk(OG_DATA_DIR))[1]
print(subfolders, )

In [ ]:
img_height, img_width = 64, 64
batch_size = 128
target_size = (64, 64)

# Load the training dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/train",
    image_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='grayscale'
)

# Load the testing dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "data/test",
    image_size=(img_height, img_width),
    batch_size=4,
    color_mode='grayscale'
)

In [ ]:
model = Sequential([ 
    layers.Rescaling(1./255, input_shape=(64,64,1)), 
    layers.Conv2D(16, 3, padding='same', activation='relu'), 
    layers.MaxPooling2D(), 
    layers.Conv2D(32, 3, padding='same', activation='relu'), 
    layers.MaxPooling2D(), 
    layers.Conv2D(64, 3, padding='same', activation='relu'), 
    layers.MaxPooling2D(), 
    layers.Flatten(), 
    layers.Dense(128, activation='relu'), 
    layers.Dense(200) 
]) 

In [ ]:
model.compile(
    optimizer="adam",
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_ds,
    validation_data=test_ds,
    epochs = 25,
    verbose=1
)

In [ ]:
import numpy

plt.figure(figsize=(10,10))
for images, labels in test_ds.take(1):
  classifications = model(images)
  # print(classifications)
  
  for i in range(4):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    index = numpy.argmax(classifications[i])
    plt.title("Pred: " + subfolders[index] + " | Real: " + subfolders[labels[i]])

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model_128.tflite", 'wb') as f:
  f.write(tflite_model)

In [ ]:
model.evaluate(test_ds)

In [ ]:
from keras.utils import load_img, img_to_array

target_size = (64, 64)

# Load and preprocess the image
image_path = 'data/test/2654/8.jpg'
image = load_img(image_path, target_size=target_size, color_mode="grayscale")
# image.save('data/test_3002 (2).jpeg')

# image = Image.open('data/test_3002.jpeg').convert('L')
image_array = img_to_array(image)
image_array = image_array / 255.0  # Normalize pixel values
plt.imshow(image_array)
# Add an extra dimension to the image array to match the input shape expected by the model
image_array = np.expand_dims(image_array, axis=0)
print(image_array.shape)

# Perform inference
predictions = model.predict(image_array)

# Get top 3 predicted classes and their probabilities
top_classes_idx = np.argsort(predictions[0])[-3:][::-1]  # Indices of top 3 classes
top_classes_prob = predictions[0][top_classes_idx]  # Probabilities of top 3 classes
top_classes_labels = [subfolders[i] for i in top_classes_idx]  # Labels of top 3 classes

# Show the top 3 predicted classes and their probabilities
for label, prob in zip(top_classes_labels, top_classes_prob):
    print(f"Class: {label}, Probability: {prob:.4f}")